In [1]:
from sandbox.qemu import QemuVMManager, VMConfig

In [2]:
vm_config = VMConfig(
    container_name="TestVM",
    base_vm_path="./docker/vms/ubuntu-noble",
    shared_path="./docker/shared",
)


manager = QemuVMManager(vm_config)
manager.get_or_create_container()

2025-04-17 10:20:05,586 - QemuVMManager - INFO - VM state changed: created -> created


2025-04-17 10:20:33,099 - QemuVMManager - INFO - Creating container TestVM with storage at /home/smwhering/IS-master-thesis/src/docker/environments/TestVM_1744878005
2025-04-17 10:20:33,100 - QemuVMManager - INFO - Binding host docker/shared/TestVM_1744878005 → container /shared docker/shared/TestVM_1744878005
2025-04-17 10:20:33,101 - QemuVMManager - INFO - Checking image qemux/qemu locally...
2025-04-17 10:20:33,105 - QemuVMManager - INFO - Image qemux/qemu exists.
2025-04-17 10:20:33,146 - QemuVMManager - INFO - Container TestVM created
2025-04-17 10:20:33,147 - QemuVMManager - INFO - VM state changed: created -> starting
2025-04-17 10:20:33,313 - QemuVMManager - INFO - Container TestVM started
2025-04-17 10:20:33,313 - QemuVMManager - INFO - VM state changed: starting -> running
2025-04-17 10:20:33,313 - QemuVMManager - INFO - SSH connection attempt 1/5...
2025-04-17 10:20:33,314 - QemuVMManager - INFO - Connection attempt failed: [Errno None] Unable to connect to port 2222 on 127.

<Container: dc9cf5d37ac0>

In [3]:
manager.cleanup_vm()

2025-04-17 10:21:18,548 - QemuVMManager - INFO - Stopping container...
2025-04-17 10:21:19,954 - QemuVMManager - INFO - Removing container...
2025-04-17 10:21:19,966 - QemuVMManager - INFO - Container removed.
2025-04-17 10:21:19,967 - QemuVMManager - INFO - Deleting instance storage directory: /home/smwhering/IS-master-thesis/src/docker/environments/TestVM_1744877700
2025-04-17 10:21:20,417 - QemuVMManager - INFO - Deleted instance storage: /home/smwhering/IS-master-thesis/src/docker/environments/TestVM_1744877700
2025-04-17 10:21:20,418 - QemuVMManager - INFO - Deleting instance storage directory: /home/smwhering/IS-master-thesis/src/docker/environments/TestVM_1744878005
2025-04-17 10:21:21,161 - QemuVMManager - INFO - Deleted instance storage: /home/smwhering/IS-master-thesis/src/docker/environments/TestVM_1744878005
